<img align="left" src="https://panoptes-uploads.zooniverse.org/project_avatar/86c23ca7-bbaa-4e84-8d8a-876819551431.png" type="image/png" height=100 width=100>
</img>


<h1 align="right">KSO Tutorials #19: Process Spyfish Aotearoa videos</h1>
<h3 align="right">Written by @jannesgg and @vykanton</h3>
<h5 align="right">Last updated: Nov 14th, 2021</h5>

# Set up and requirements

Import Python packages

In [ ]:
# Set the directory of the libraries
import sys
sys.path.append('..')

# Set to display dataframes as interactive tables
from itables import init_notebook_mode
init_notebook_mode(all_interactive=True)

# Import required modules
import kso_utils.tutorials_utils as t_utils
import kso_utils.t19_utils as t19
#import utils.server_utils as serv_utils

print("Packages loaded successfully")

### Choose your project

In [ ]:
project = t_utils.choose_project()

### Initiate sql and get server or local storage details

In [ ]:
# Initiate db
db_info_dict = t_utils.initiate_db(project.value)

# Select the folder where the sd card is

Make sure you select the FOLDER not the files

In [ ]:
go_pro_folder = t19.select_go_pro_folder()

In [ ]:
go_pro_movies_i = t19.select_go_pro_movies(go_pro_folder.selected)
print("The movies selected are:")
print(*go_pro_movies_i, sep='\n')

### Select the date and site

In [ ]:
site_i = t19.select_site(db_info_dict)
date_i = t19.select_date()

### Concatenate the go pro files

In [ ]:
video_info_dict = t19.concatenate_go_pro_videos(site_i.result.value,
                                                date_i.value.isoformat().replace("-","_"),
                                                go_pro_folder.selected,
                                                go_pro_movies_i)

### Select Author, bad deployment, survey start/end, s3 bucket

In [ ]:
# Select author
video_author = t19.select_author(db_info_dict)

# Specify id deployment is bad
deployment_i = t19.select_bad_deployment()

# Set survey start
surv_start = t19.select_start_survey(video_info_dict["length"])

#TODO Set survey end
surv_end = t19.select_end_survey(video_info_dict["length"])

# Select the S3 "folder" to upload the video
s3_folder = t19.select_s3_folder(db_info_dict)

#Add any comment related to the movie
comment_movie = t19.write_comment()

### Review the survey/movie details

In [ ]:
new_row, movies_df = t19.review_movie_details(
    project_name = project.value,
    db_info_dict = db_info_dict,
    video_info_dict_i = video_info_dict,
    IsBadDeployment_i = deployment_i.result,
    survey_start_i = surv_start.result,
    survey_end_i = surv_end.result,
    author_i = video_author.result.value,
    comment_i = comment_movie.value,
    s3_prefix_i = s3_folder.value,
    go_pro_list_i = go_pro_movies_i,
    bucket_i = 'marine-buv'
)
print("The new details are:")
new_row.to_dict('records')

# !!!Only pass this point if movie details are correct!!!

### Update movies csv and upload video to s3

In [ ]:
upload_concat_movie(new_row, db_info_dict, video_info_dict, movies_df)

In [ ]:
#END